# Motion Planning, Assignment 5: Rapidly Exploring Random Trees

#### Thao Dang, Hochschule Esslingen

<a id="fig_1"></a>
![Fig. 1: Robot scenario](https://drive.google.com/uc?export=view&id=1DWaUj5gBk9fZm9SEzFPan4VKp-cmGVwK)

In this assignment, you will search an collision-free path with an Rapidly Exploring Random Tree. The algorithm is described in the lecture slides. Please note that you will be using goal-oriented sampling.

The robot motion will be executed by a simple local planning function that allows the robot to move forward in any direction but stops when the robot encounters an obstacle.

First, the usual boilerplate code to load the necessary libraries:

In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

The following defines our world including start and goal configurations. Obstacles are defined as circles with various radii. A few helper functions are given to draw the scene.

In [ ]:
def plot_circle(x, y, size, color="-b"):
    """Draw circle of radius size and center pos (x,y)."""
    deg = list(range(0, 360, 5))
    deg.append(0)
    xl = [x + size * np.cos(np.deg2rad(d)) for d in deg]
    yl = [y + size * np.sin(np.deg2rad(d)) for d in deg]
    plt.plot(xl, yl, color)


def plot_obstacles(obstacle_list, q_start, q_end):
    """Draw all obstacles in the scene."""
    for (ox, oy, size) in obstacle_list:
        plot_circle(ox, oy, size)

    plt.plot(q_start[0], q_start[1], "xg", label="start")
    plt.plot(q_end[0], q_end[1], "sr", label="goal")

    plt.axis([-2, 15, -2, 15])
    plt.grid(True)
    plt.legend(loc="upper right")


obstacle_list = [
    (5, 5, 1),
    (3, 6, 2),
    (3, 8, 2),
    (3, 10, 2),
    (7, 5, 2),
    (9, 5, 2),
    (8, 10, 1),
]  # [x, y, radius]

q_start_pos, q_end_pos = (0, 0), (6, 10)

plt.figure(figsize=(8, 8))
plot_obstacles(obstacle_list, q_start_pos, q_end_pos)

## Exercise 1: Finding nearest nodes and checking collisions

Below, you will find a basic class for a node in the random tree structure. Please note that the node structure contains the position of the node, its parent, and its ``path``. The ``path`` reflects that a local planner will be used later on and stores the sampled path from the parent position to the current position of the node.

Implement two functions ``get_nearest_node_index`` and ``check_collision`` below. ``get_nearest_node_index`` shall find the index of closest known node in the graph to a given refernce position. ``check_collision``
will return ``True`` if a given positions collides with the obstacles.

More information on the syntax of these functions is given in the docstrings below. Also make sure that your code passes the tests.

In [ ]:
class Node:
    """RRT Node

    The attributes of the class are
    * pos: the position of the node, format: tuple "(x,y)"
    * path: the sampled path of the robot from the parent position to the
            current position, format: list of tuples "[(x,y), ...]".
            NOTE: the path shall include both the parent position
                  "parent.pos" as well as the current positon "pos"
    * parent: the parent node, format: RRT node "node"
    """

    def __init__(
        self,
        pos: tuple[float, float],
        path: list[tuple[float, float]] = [],
        parent=None,
    ):
        self.pos = pos
        self.path = path
        self.parent = parent

    def __str__(self):
        """Allows printing of a node."""
        return (
            "<"
            + str(self.pos)
            + ", path="
            + str(self.path)
            + ", parent="
            + str(self.parent.pos)
            + ">"
        )


def get_nearest_node_index(node_list: Node, rnd_node_pos: tuple[float, float]):
    """Return index of closest node to position "rnd_node_pos" in list of nodes "node_list"."""
    rnd_node_pos = pd.Series(rnd_node_pos, index=["x", "y"])
    node_pos = pd.DataFrame((n.pos for n in node_list), columns=["x", "y"])
    node_dist = node_pos.apply(
        lambda pos: np.linalg.norm(pos - rnd_node_pos, ord=2), axis="columns"
    )
    return node_dist.argmin()


def check_collision(
    node_pos: tuple[float, float], obstacle_list: list[tuple[float, float, float]]
):
    """Check if "node_pos" collides with any of the obstacles in "obstacle_list".

    Returns "True" if a collision has been detected, "False" otherwise.
    """
    nx, ny = node_pos
    npos = np.array([nx, ny])
    return any(
        np.linalg.norm(npos - np.array([cx, cy])) < cr for (cx, cy, cr) in obstacle_list
    )


def calc_distance_and_angle(from_node, to_node):
    """Compute distance and angle (in rad) from "from_node" to "to_node"."""
    dx = to_node.pos[0] - from_node.pos[0]
    dy = to_node.pos[1] - from_node.pos[1]
    d = np.sqrt(dx**2 + dy**2)
    theta = np.arctan2(dy, dx)
    return d, theta


## tests
n1 = Node((1, 1), path=[(0, 0), (0.3, 0.3), (0.6, 0.6), (1, 1)], parent=None)
n2 = Node((2, 2), path=[(1, 1), (1.3, 1.3), (1.6, 1.6), (2, 2)], parent=n1)
n3 = Node((8, 8), path=[(2, 2), (4, 4), (6, 6), (8, 8)], parent=n2)
n4 = Node((6, 6), path=[(2, 2), (3, 3), (4, 4), (5, 5), (6, 6)], parent=n2)
rnd_node_pos = (3, 2)
node_list = [n1, n2, n3, n4]

assert get_nearest_node_index(node_list, rnd_node_pos) == 1

assert not check_collision(n1.pos, obstacle_list)
assert check_collision(n4.pos, obstacle_list)

assert calc_distance_and_angle(n1, n2) == (np.sqrt(2), np.pi / 4.0)

## Excercise 2: Local planner

The core of the RRT is the local planner. We assume that in each step, the robot can move forward
* in a given direction ``theta`` and 
* at most about a length ``stepSize*numSteps``.

The local planner will move the robot from a given start node ``from_node`` towards a desired node ``to_node``. While moving forward, it will check for collisions at every distance increment ``stepSize``. When a collision is detected, the robot will remain at its last valid position and the function will return a corresponding node.

When the robot reaches a distance to the desired node ``to_node`` that is below ``stepSize``, the robot shall move to the desired position and return a corresponding node.

Implement the function below and also add some tests when the local planner shall stop in front of an obstacle.

In [ ]:
def local_planner(
    from_node: Node,
    to_node: Node,
    obstacle_list: list[tuple[float, float, float]],
    stepSize=0.33,
    numSteps=3,
):
    """Local planner function.

    Arguments:
        * from_node: the start node (type: class Node)
        * to_node: the target node (type: class Node)
        * obstacle_list: the circular obstacles in the scene (type: [(x,y,r), ...])
        * stepSize: the robot should move about this distance for "numSteps" times
            and check collisions on each individual step (type: float)
        * numSteps: the number of steps to make (type: int)

    Returns:
        The resulting new node (type: class Node). Note that the robot shall move **exactly**
        to the desired node if the distance between the robot and "to_node" is below "stepSize".
    """
    to_x, to_y = to_node.pos

    new_node = Node(from_node.pos, path=[from_node.pos], parent=from_node)

    for _ in range(numSteps):
        cur_x, cur_y = new_node.pos
        prev_path = new_node.path
        if np.linalg.norm(np.array([to_x - cur_x, to_y - cur_y]), ord=2) < stepSize:
            new_pos = to_node.pos
        else:
            theta = np.arctan2(to_y - cur_y, to_x - cur_x)
            new_pos = (
                cur_x + np.cos(theta) * stepSize,
                cur_y + np.sin(theta) * stepSize,
            )
        if check_collision(new_pos, obstacle_list):
            break
        prev_path.append(new_pos)
        new_node = Node(new_pos, path=prev_path, parent=new_node)

    return new_node


q_new = local_planner(n1, n2, obstacle_list, stepSize=0.5, numSteps=3)
assert q_new.pos == n2.pos
assert q_new.path[0] == n1.pos
assert q_new.path[-1] == n2.pos
assert len(q_new.path) == 4

## Excercise 2: Local planner

Using all the helper functions defined above, implement a goal oriented RRT as described in the lecture notes.

In [ ]:
def draw_graph(node_list, result_path=None):
    """Draws the graph and found path."""
    for node in node_list:
        path_x, path_y = zip(*node.path)
        plt.plot(path_x, path_y, "-g")

    if result_path:
        path_x, path_y = zip(*result_path)
        plt.plot(path_x, path_y, "-r")


def RRT(
    q_start_pos,
    q_end_pos,
    obstacle_list,
    goal_sampling_rate=0.1,
    worldSize=(15, 15),
    maxIter=1000,
):
    """Goal oriented RRT.

    Arguments:
        * q_start_pos: the start position (type: tuple (x,y))
        * q_end_pos: the goal position (type: tuple (x,y))
        * obstacle_list: the circular obstacles in the scene (type: [(x,y,r), ...])
        * goal_sampling_rate: the rate with which the goal position is drawn as a
            new desired node (type: float)
        * worldSize: the size of the world used for sampling new positons (type: tuple (max_x,max_y))
        * maxIter: the maximum number of iterations used in the RRT algorithm (type: int)

    Returns:
        * path: the path from start to goal ("None" if none was found, a list of positions
            [(x,y), ...] otherwise.
        * node_list: the list of nodes that forms the random tree (type: [class Node])
    """

    q_start = Node(q_start_pos, path=[q_start_pos], parent=None)
    node_list = [q_start]

    for _ in range(maxIter):
        if random.random() < goal_sampling_rate:
            q_rand_pos = q_end_pos
        else:
            q_rand_pos = (
                np.random.uniform(0, worldSize[0]),
                np.random.uniform(0, worldSize[1]),
            )

        if check_collision(q_rand_pos, obstacle_list):
            continue

        q_rand = Node(q_rand_pos)
        i_next = get_nearest_node_index(node_list, q_rand_pos)
        q_near = node_list[i_next]
        q_new = local_planner(q_near, q_rand, obstacle_list)
        node_list.append(q_new)

        fx, fy = q_new.pos
        tx, ty = q_end_pos

        if np.linalg.norm(np.array([fx - tx, fy - ty]), ord=2) < 0.0001:
            break

    return q_new.path, node_list


# find path to goal
path, node_list = RRT(q_start_pos, q_end_pos, obstacle_list, maxIter=1000)

# plot results
plt.figure(figsize=(8, 8))
plot_obstacles(obstacle_list, q_start_pos, q_end_pos)
draw_graph(node_list, result_path=path)